---

# <center>  팀 인사이트 모델 코드 </center>

---

## <center> 개발환경 및 라이브러리 정보 </center>

모듈(OS) | 버전
--------- | ---------
OS | macOS Catalina
python & 내장 모듈 | 3.8.3 64-bit
tqdm | 4.56.0
pandas | 1.2.1
numpy | 1.20.0
catboost | 0.24.4
sklearn | 0.0

---

# <center>목차</center>

* `Index`
    - **1. 데이터셋 준비**
        - 1-1. 라이브러리 / 데이터 로드  
        - 1-2. 결측치 처리
    - **2. 데이터 전처리**
        - 2-1. 변수 처리
        - 2-2. 파생변수 생성
            - 2-2-1. make_dataset1 : 에러타입, 주요 에러코드 관련 변수
            - 2-2-2. make_dataset2 : 단위시간 별 에러타입/퀄리티 관련 변수
            - 2-2-3. make_dataset3 : 주로 펌웨어 버전, 모델 관련 변수
            - 2-2-4. 최종 데이터 병합 <br>
    - **3. 모델 훈련 및 예측**
        - 3-1. 모델 학습
        - 3-2. 예측 및 제출파일 생성 <br>

---

# <center>1. 데이터셋 준비</center>

---

## 1-1. 라이브러리 / 데이터 로드

In [ ]:
from tqdm import tqdm

import pandas as pd
import numpy as np

import datetime as dt
import collections
import random
from catboost import CatBoostRegressor

from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore') 

DATA_PATH = "./preprocessing/"
SAVE_PATH = "./preprocessing/"
RESULT_PATH = "./result/"

### 기본 데이터
- train_err : train_err_data
- train_qual : train_quality_data
- train_problem : train_problem_data  
<br>
- test_err : test_err_data
- test_qual : test_quality_data  
<br>
- problem_user : 불만을 제기한 고객들
- no_problem_user : 불만을 제기안한 고객들

In [ ]:
# 데이터 로드
train_err = pd.read_csv(DATA_PATH+ 'train_err_data.csv')
train_qual = pd.read_csv(DATA_PATH+"train_quality_data.csv")
train_problem = pd.read_csv(DATA_PATH+ 'train_problem_data.csv')

test_err = pd.read_csv(DATA_PATH+"test_err_data.csv")
test_qual = pd.read_csv(DATA_PATH+'test_quality_data.csv')

# 불만 접수 고객 확인
problem_user = train_problem.user_id.unique()
no_problem_user = list(set(train_err.user_id.unique())  - set(problem_user))

In [ ]:
# 난수 고정
random.seed(1422)

## 1-2. 결측치 처리

In [ ]:
# 결측치 인덱스 슬라이싱 : EDA로 확인 후 최빈값으로 대체 
train_err.iloc[3825744,5] = train_err.errcode.mode()[0]

test_err.iloc[937967,5] = test_err.errcode.mode()[0]
test_err.iloc[4038892,5] = test_err.errcode.mode()[0]
test_err.iloc[9486881,5] = test_err.errcode.mode()[0]
test_err.iloc[10425473,5] = test_err.errcode.mode()[0]

In [ ]:
# 퀄리티 데이터는 0으로 대체
train_qual = train_qual.fillna(0)
test_qual = test_qual.fillna(0)

---

# <center>2. 데이터 전처리</center>

---

## 2-1. 변수 처리

In [ ]:
# 데이터 기본 정보
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

- pre_funcs : 데이터 전처리 함수들의 집합
  - make_datetime : 날짜 데이터를 datetime type으로 변환
  - get_total : 각 변수 별 모든 종류 리스트 반환
  - make_day : 일(date) 추가
  - make_hour : 시(hour) 추가
  - str2int : str 타입을 int 타입으로 변환

In [ ]:
class pre_funcs() : # 데이터 전처리 함수들
    
    def make_datetime(x): # datetime 데이터로 변환
        x     = str(x)
        year  = int(x[:4])
        month = int(x[4:6])
        day   = int(x[6:8])
        hour  = int(x[8:10])
        minute  = int(x[10:12])
        sec  = int(x[12:])
        return dt.datetime(year, month, day, hour,minute,sec)
    
    def get_total(col,df1,df2): # 각 데이터별 모든 종류구하기
        train_unique = set(df1[col].unique())
        test_unique  = set(df2[col].unique())
        total = (train_unique | test_unique) 
        return total
    
    def make_day(col): # 일자 반환
        col = str(col)
        year  = int(col[:4])
        month = int(col[4:6])
        day   = int(col[6:8])

        if month == 10:
            return 0
        elif month == 11:
            return day
        else:
            return 32
    
    def make_hour(col): # 시 반환
        return int(str(col)[8:10])
    
    def str2int(x): # string -> int
        if type(x) == str:
            x = x.replace(",","")
            x = float(x)
            return int(x)
        else:
            return int(x)

In [ ]:
# 파생변수 목적에 맞게 각기 다른 전처리 : err1 -> mk_dataset1 
train_err1 = train_err.copy()
train_err1['date_k'] = train_err1['time'].apply(lambda x:pre_funcs.make_day(x))
train_err1['hour'] = train_err1['time'].apply(lambda x:pre_funcs.make_hour(x))

test_err1 = test_err.copy()
test_err1['date_k'] = test_err1['time'].apply(lambda x:pre_funcs.make_day(x))
test_err1['hour'] = test_err1['time'].apply(lambda x:pre_funcs.make_hour(x))

In [ ]:
# 파생변수 목적에 맞게 각기 다른 전처리 : err2 -> mk_dataset3
train_err2 = train_err.copy()
test_err2 = test_err.copy()

train_err2.time = train_err2.time.apply(lambda x: pre_funcs.make_datetime(x))
test_err2.time = test_err2.time.apply(lambda x: pre_funcs.make_datetime(x))

model_total= pre_funcs.get_total('model_nm',train_err2,test_err2)
errtype_total=pre_funcs.get_total('errtype',train_err2,test_err2)
fwver_total = pre_funcs.get_total('fwver',train_err2,test_err2) 

In [ ]:
# quality 

# qual1 : train 
qual1 = train_qual.copy()
qual1['date'] = qual1['time'].apply(lambda x: int(str(x)[:8]))
list_qual = list(qual1.columns[3:-1])
for i in list_qual:
    qual1[i] = qual1[i].apply(lambda x: pre_funcs.str2int(x))
    
# qual2 : test
qual2 = test_qual.copy()
qual2['date'] = qual2['time'].apply(lambda x: int(str(x)[:8]))
list_qual2 = list(qual2.columns[3:-1])
for i in list_qual:
    qual2[i] = qual2[i].apply(lambda x: pre_funcs.str2int(x))

## 2-2. 파생변수 생성

### 2-2-1. make_dataset1
  - run : 전체 함수 실행
  - make_errcode_list : 주요 에러코드 리스트 생성
  - df_make : 에러타입/에러코드마다 카운트, 날짜 및 시간대 별 전체 에러타입 발생횟수

In [ ]:
class make_dataset1 : # 에러타입/ 에러코드 관련 파생변수 생성 
    
    def __init__(self, df, types, num_user, min_idx):
        self.df = df
        self.types = types
        self.num_user = num_user
        self.min_idx = min_idx
        
    def run(self) : # 전체 실행 함수
        self.critical_errcode = self.make_errcode_list()
        result = self.df_make()
        return result
        
    def make_errcode_list (self): # 주요 에러코드 리스트 생성 함수
        label = np.zeros(15000)
        label[train_problem.user_id.unique()-10000] = 1 

        problem_list = list(problem_user)
        problem_code= train_err[train_err['user_id'].isin(problem_list)]['errcode'].value_counts().to_dict()
        no_problem_code = train_err[~train_err['user_id'].isin(problem_list)]['errcode'].value_counts().to_dict()

        total_code = []
        critical_errcode = []
        for i in problem_code:
            try :
                total_code.append((i,problem_code[i]/(problem_code[i]+no_problem_code[i]),problem_code[i],no_problem_code[i]))
            except :
                total_code.append((i,1,problem_code[i],0))

        for i in no_problem_code:
            if i in problem_code:
                continue
            total_code.append((i,0,0,no_problem_code[i]))

        total_code.sort(key = lambda x: x[1], reverse = True)


        for i in total_code:
            if i[2] >= 5 or i[3] >= 5:
                critical_errcode.append(i[0])
        return critical_errcode

    def df_make (self) : # 에러코드, 날짜, 시간대 데이터 생성 함수
        num_user = self.num_user
        min_idx = self.min_idx
        clist = self.critical_errcode

        id_error = self.df[['user_id','errtype','errcode', 'fwver', 'date_k', 'hour']].values
        error = np.zeros((num_user,223))
        vers = [' '] * num_user

        # 0~ 41 : 에러타입, 42~165: 에러코드분류, 166~198: 날짜, 199~222 : 시간대
        for person_idx, err, code, v, d, h in tqdm(id_error):
            error[person_idx - min_idx,err - 1] += 1

            if code in clist :
                error[person_idx - min_idx,42+clist.index(code)] += 1

            vers[person_idx - min_idx] = v

            error[person_idx - min_idx,166 + d ] += 1
            error[person_idx - min_idx,199 + h ] += 1

        res = pd.DataFrame(data=error, columns = 
                ['err_' + str(i+1) + '_cnt' for i in range(42)]+
                ['err_' + str(i+1) + '_code' for i in range(124)]+
                ['err_' + str(i+1) + '_date' for i in range(33)]+
                ['err_' + str(i+1) + '_hour' for i in range(24)])
        res['Total'] = res.sum(axis=1)
        res['ver'] = vers
        return res

In [ ]:
train1 = make_dataset1(train_err1,'train', 15000, 10000).run()
test1 = make_dataset1(test_err1,'test', 14999, 30000).run()

### 2-2-2. make_dataset2
  - make_qual_data : 퀄리티 관련 함수들 모두 실행
  - move_errtype : 단위 기간별(3일,7일) 에러타입 통계수치
  - qual_cnt : 퀄리티마다 단순합
  - make_id_in_qual : 퀄리티 데이터에 없는 id 채우기
  - move_quality : 12행마다 퀄리티 관련 통계수치

In [ ]:
class make_dataset2():# 에러타입 파생변수, 퀄리티 파생변수 생성
    def __init__(self, df, scope, num_user, min_idx):
        self.df = df
        self.scope = scope
        self.num_user = num_user
        self.min_idx = min_idx
    
    def make_qual_data(self): # 퀄리티 관련 함수 실행
        qual_plus = self.qual_cnt()
        
        qual_tmp = self.make_id_in_qual()
        qual_tmp0 = self.move_quality(qual_tmp, 0, moving=12)
        qual_tmp1 = self.move_quality(qual_tmp, 1, moving=12)
        qual_tmp2 = self.move_quality(qual_tmp, 2, moving=12)
        qual_tmp3 = self.move_quality(qual_tmp, 3, moving=12)
        qual_tmp4 = self.move_quality(qual_tmp, 4, moving=12)
        qual_tmp5 = self.move_quality(qual_tmp, 5, moving=12)
        qual_tmp6 = self.move_quality(qual_tmp, 6, moving=12)
        qual_tmp7 = self.move_quality(qual_tmp, 7, moving=12)
        qual_tmp8 = self.move_quality(qual_tmp, 8, moving=12)
        qual_tmp9 = self.move_quality(qual_tmp, 9, moving=12)
        qual_tmp10 = self.move_quality(qual_tmp, 10, moving=12)
        qual_tmp11 = self.move_quality(qual_tmp, 11, moving=12)
        qual_tmp12 = self.move_quality(qual_tmp, 12, moving=12)
        
        return pd.concat([qual_plus, qual_tmp0, qual_tmp1, qual_tmp2, qual_tmp3, 
                        qual_tmp4, qual_tmp5, qual_tmp6, qual_tmp7, qual_tmp8, qual_tmp9, qual_tmp10,
                        qual_tmp11, qual_tmp12], axis=1).fillna(0)
    
    
    def move_errtype(self): # 단위 시간별 에러타입 통계수치
        tqdm.pandas()
        data = self.df
        data2 = data.copy()
        scope= self.scope
        data2['date'] = data['time'].apply(lambda x: int(str(x)[:8]))

        # user_id, date, errtype
        errtype_array = np.zeros((self.num_user, 30, 42))
        user_id_min = min(data2['user_id'])

        def myfunc(x):
            for day in range(x['date'] % 100, min(31, x['date'] % 100 + scope)):
                errtype_array[x['user_id'] - user_id_min, day-1, x['errtype']-1] += 1

        data2.progress_apply(myfunc, axis=1)

        errtype_array_max = errtype_array.max(axis=1)
        errtype_array_min = errtype_array.min(axis=1)
        errtype_array_std = errtype_array.std(axis=1)

        return pd.concat([pd.DataFrame(data=errtype_array_max, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_max' for i in range(42)]),
                          pd.DataFrame(data=errtype_array_min, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_min' for i in range(42)]), 
                          pd.DataFrame(data=(errtype_array_max - errtype_array_min), columns=['err_' + str(i+1) + 
                                                                                              '_move_' + str(scope) + 
                                                                                              '_diff' for i in range(42)]), 
                          pd.DataFrame(data=errtype_array_std, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_std' for i in range(42)])], axis=1)
    def qual_cnt(self) : # 퀄리티 단순합
        id_q = self.df[['user_id','fwver',"quality_0","quality_1","quality_2","quality_3","quality_4","quality_5","quality_6","quality_7","quality_8","quality_9","quality_10","quality_11","quality_12"]].values
        qual = np.zeros((self.num_user,41))
        
        # 0~ 27 : 펌웨어, 28~40: 퀄리티
        for person_idx, fv, q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12 in tqdm(id_q):

            qual[person_idx - self.min_idx,28] += q0
            qual[person_idx - self.min_idx,29] += q1
            qual[person_idx - self.min_idx,30] += q2
            qual[person_idx - self.min_idx,31] += q3
            qual[person_idx - self.min_idx,32] += q4
            qual[person_idx - self.min_idx,33] += q5
            qual[person_idx - self.min_idx,34] += q6
            qual[person_idx - self.min_idx,35] += q7
            qual[person_idx - self.min_idx,36] += q8
            qual[person_idx - self.min_idx,37] += q9
            qual[person_idx - self.min_idx,38] += q10
            qual[person_idx - self.min_idx,39] += q11
            qual[person_idx - self.min_idx,40] += q12
            
        qual_plus = pd.DataFrame(data=qual, columns = ['err_'+str(i+1)+'_qual' for i in range(41)])
        qual_plus = qual_plus.iloc[:,28:]
        qual_plus.columns = ['err_'+str(i)+'_qual' for i in range(13)]
        return qual_plus
    
    def make_id_in_qual(self): # 퀄리티 데이터에 없는 id 채우기
        data = self.df
        data2 = data.copy()
        lst_id1 = sorted(list(data2.user_id.unique()))

        for i in tqdm(range(self.min_idx, self.num_user +self.min_idx)):
            if i in lst_id1:
                pass
            elif i not in lst_id1:
                tmp_d = {'time': 20201101090000,
                         'user_id': i,
                         'fwver': '04.11.1384',
                         'quality_0':0,'quality_1':0,'quality_2':0,'quality_3':0,'quality_4':0,'quality_5':0,
                         'quality_6':0,'quality_7':0,'quality_8':0,'quality_9':0,'quality_10':0,'quality_11':0,
                         'quality_12':0}
                data2 = data2.append(tmp_d, ignore_index=True)

        return data2
    
    def move_quality(self, data, number, moving=12): # 퀄리티 통계수치
        tqdm.pandas()
        

        # max
        max_data = data.groupby(['user_id'])['quality_' + str(number)].rolling(window=moving).max()
        max_data = max_data.to_frame()
        max_data = max_data.groupby(['user_id'])['quality_' + str(number)].max().to_frame().reset_index()
        max_data = max_data.iloc[:,1:]
        
        # min
        min_data = data.groupby(['user_id'])['quality_' + str(number)].rolling(window=moving).min()
        min_data = min_data.to_frame()
        min_data = min_data.groupby(['user_id'])['quality_' + str(number)].min().to_frame().reset_index()
        min_data = min_data.iloc[:,1:]
        
        # std
        std_data = data.groupby(['user_id'])['quality_' + str(number)].rolling(window=moving).std()
        std_data = std_data.to_frame()
        std_data = std_data.groupby(['user_id'])['quality_' + str(number)].mean().to_frame().reset_index()
        std_data = std_data.iloc[:,1:]
        
        # mean
        mean_data = data.groupby(['user_id'])['quality_' + str(number)].rolling(window=moving).mean()
        mean_data = mean_data.to_frame()
        mean_data = mean_data.groupby(['user_id'])['quality_' + str(number)].mean().to_frame().reset_index()
        mean_data = mean_data.iloc[:,1:]

        # concat
        moving_result = pd.concat([max_data, min_data, std_data, mean_data], axis=1)
        moving_result.columns = ['quality_'+str(number)+'_max',
                                 'quality_'+str(number)+'_min', 'quality_'+str(number)+'_std',
                                 'quality_'+str(number)+'_mean']

        return moving_result
    

In [ ]:
train2 = make_dataset2(train_err,3,15000,10000).move_errtype()
train3 = make_dataset2(train_err,7,15000,10000).move_errtype()
train4 = make_dataset2(qual1,7,15000,10000).make_qual_data()

In [ ]:
test2 = make_dataset2(test_err,3,14999,30000).move_errtype()
test3 = make_dataset2(test_err,7,14999,30000).move_errtype()
test4 = make_dataset2(qual2,7,14999,30000).make_qual_data()

### 2-2-3. make_dataset3
  - run : 모든 함수 실행
  - errtype_cnt : 전체 에러타입 발생횟수
  - model_change : 모델의 변화
  - fw_change : 펌웨어 버전의 변화
  - errtype_sta : 전체 에러타입 발생횟수 관련 통계수치
  - errtype_of_hour : 시간별 에러타입 통계수치
  - errtype_of_day : 날짜별 에러타입 통계수치
  - fwver_flows : 펌웨어버전 변경 패턴
  - get_time_term : 시간 차이 
  - connetion_err : connetion_err 에러코드 발생횟수
  - fwver_time : 가자 오래 사용한 버전 관련 변수
  - fw_err_cnt : 해당 버전과 에러타입 관련 변수

In [ ]:
class mk_dataset3(): # train_err_data 관련 파생변수 생성 
    
    def __init__(self, df, types, num_user, num_errtype, first):
        self.df = df
        self.types = types
        self.num_user = num_user
        self.num_errtype = num_errtype
        self.first = first
        self.num_model_change =2
        
    def run(self):
        vrb1 = self.errtype_cnt() # 에러타입마다 카운트
        vrb2 =self.model_change() # 모델 변화
        vrb3 =self.fw_change() # 펌웨어 버전 변화
        m, std =self.errtype_sta() # 주별 에러타입 통계수치
        mhour, stdhour, maxhour =self.errtype_of_hour() # 시간별 에러타입 통계수치
        mday, stdday, maxday =self.errtype_of_day() # 일별 에러타입 통계수치
        vrb4 =self.fwver_flows() # 펌웨어버전 연관성
        
        
        
        vrb5 =self.get_time_term() # 시간별 에러발생 상황
        vrb6 =self.connetion_err() # connetion_err
        vrb7 =self.fwver_time() # 펌웨어 버전 변경까지의 시간
        vrb8=self.fw_err_cnt() # 주요 펌웨어 버전 카운트
        
        
        return [vrb3,vrb2, vrb1, m, std ,mhour, stdhour, maxhour,mday, maxday, stdday, vrb4 , vrb5 ,vrb6  ,vrb7,vrb8]
        
    def errtype_cnt (self) : 
        errtypes = np.zeros((self.num_user,self.num_errtype))

        for idx, value in tqdm(self.df[['user_id','errtype']].values):
            errtypes[idx-self.first,value-1] +=1
        return errtypes


    def model_change(self): 
        df = self.df
        first = self.first
        
        v=df[['user_id','model_nm']]
        getdf =~(v == v.shift(1))
        logical =(getdf.user_id.apply(int) + getdf.model_nm.apply(int)) > 0
        df_model_counts =v[logical]


        def get_model_change_id(num):# 모델이 변화한 유저
            df_mc = df_model_counts.user_id.value_counts()
            df_mc_user=list(df_mc.loc[df_mc ==num].to_frame().index)
            sort_mc_user = df.loc[df.user_id.isin(df_mc_user)].drop_duplicates(
                ['user_id','model_nm'],keep='last').drop_duplicates('user_id', keep='first').sort_values("time").user_id
            return sort_mc_user.values

        one_m = get_model_change_id(1)
        two_m = get_model_change_id(2)
        thr_m = get_model_change_id(3)


        model_n = np.zeros((self.num_user,self.num_model_change))
        df['model_f'] =df['model_nm'].str[-1].astype('int')
        one_df = df.loc[df.user_id.isin(one_m)][['user_id','model_f']].drop_duplicates().values
        two_df =df.loc[df.user_id.isin(two_m)][['user_id','model_f']].drop_duplicates().reset_index(drop=True)
        two_df['tf'] = two_df.index%2
        two_df= two_df.pivot(index='user_id',columns='tf').reset_index().values
        thr_df = df.loc[df.user_id.isin(thr_m)][['user_id','model_f']].drop_duplicates().values

        for inx, value in tqdm(one_df):
            model_n[inx-self.first,0]  +=value
        for inx, value1,value2 in tqdm(two_df):
            model_n[inx-self.first,0]  +=value1
            model_n[inx-self.first,1]  +=value2

        for inx, value in tqdm(thr_df):
            model_n[inx-self.first,0]  +=value
            
        return model_n

    def fw_change(self): 
        df= self.df
        
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1


        df['ver_num'] = df['fwver'].apply(lambda x : fwver_total_dic[x])
        fwver_np = np.zeros((self.num_user,5))

        v=df[['user_id','ver_num']]
        getdf =~(v == v.shift(1))
        logical =(getdf.user_id.apply(int) + getdf.ver_num.apply(int)) > 0
        fwver_num=v[logical]
        
        fwver_num = fwver_num.reset_index(drop=True)
        count =np.zeros(len(fwver_num),dtype=int)

        for v in range(1,len(fwver_num)):
            if fwver_num.user_id.values[v-1] ==fwver_num.user_id.values[v]:
                count[v] = count[v-1] +1


        fwver_num['count'] =count
        fw_v = fwver_num.loc[fwver_num['count'].isin([0,1,2,3,4])].pivot(
            index='user_id',columns='count').reset_index().fillna(0).values
        fw_v =fw_v.astype('int64')

        for inx, v1,v2,v3,v4,v5 in tqdm(fw_v):
            fwver_np[inx-self.first,0] =v1
            fwver_np[inx-self.first,1] =v2
            fwver_np[inx-self.first,2] =v3
            fwver_np[inx-self.first,3] =v4
            fwver_np[inx-self.first,4] =v5
        return fwver_np

    def errtype_sta(self):
        df = self.df
        
        df['week'] =self.df.time.dt.isocalendar().week

        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','week']]
        df_=datas[['user_id','week','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','week']).rename(columns = {0:'counts'}).reset_index(drop=True)

        df1 =df_.loc[df_.week ==44][['user_id','errtype','counts']].values
        df2 =df_.loc[df_.week ==45][['user_id','errtype','counts']].values
        df3 =df_.loc[df_.week ==46][['user_id','errtype','counts']].values
        df4 =df_.loc[df_.week ==47][['user_id','errtype','counts']].values
        df5 =df_.loc[df_.week ==48][['user_id','errtype','counts']].values

        day_data = np.zeros((self.num_user,42,5))
        for i, dfa in enumerate([df1,df2,df3,df4,df5]):
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        m=day_data.mean(axis=2)
        std=day_data.std(axis=2)    
        return m, std
        
    def errtype_of_hour (self):
        df = self.df
        df['hour'] =df.time.dt.hour


        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','hour']]
        df_=datas[['user_id','hour','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','hour']).rename(columns = {0:'counts'}).reset_index(drop=True)


        day_data = np.zeros((self.num_user,42,24))
        for i in range(24):
            dfa = df_.loc[df_['hour']==i][['user_id','errtype','counts']].values
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        mhour=day_data.mean(axis=2)
        stdhour=day_data.std(axis=2)       
        maxhour=day_data.max(axis=2)
        return mhour, stdhour, maxhour
 
    def errtype_of_day (self):
        df = self.df
        
        df['day'] =df.time.dt.day


        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','day']]
        df_=datas[['user_id','day','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','day']).rename(columns = {0:'counts'}).reset_index(drop=True)

        day_data = np.zeros((self.num_user,42,30))
        for i in range(30):
            dfa = df_.loc[df_['day']==(i+1)][['user_id','errtype','counts']].values
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        mday=day_data.mean(axis=2)
        stdday=day_data.std(axis=2)       
        maxday=day_data.max(axis=2) 
        return mday, stdday, maxday
    
    
    def fwver_flows(self):
        target_df = self.df
        first_num = self.first
        count_num =self.num_user

        dp = target_df[['user_id','model_nm','fwver']]
        unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

        dp2 = target_df[['user_id','model_nm']]
        unique_data2 =target_df[(dp2 !=dp2.shift(1)).sum(axis=1)>0]

        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1


        fwver = np.zeros((count_num,24))
        for idx in tqdm(unique_data.user_id.unique()):
            df_md =unique_data2.loc[unique_data2.user_id==idx].model_nm.values
            df_fw = unique_data.loc[unique_data.user_id==idx].fwver.values

            for md in range(len(df_md)):
                fwver[idx-first_num,md] = int(df_md[md][-1])+1

            for l in range(3,len(df_fw)+3):
                fwver[idx-first_num,l] =fwver_total_dic[df_fw[l-3]]

        fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

        fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
        fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
        fwver_total_dic_rev[0] =0
        fwver_total_dic_rev2[0] = '04.22.1750'


        fw_df[3] =fw_df[3].apply(lambda x : fwver_total_dic_rev2[x])
        for _ in range(4,8):
            fw_df[_] =fw_df[_].apply(lambda x : fwver_total_dic_rev[x])


        fw_df = fw_df.rename(columns={0:'md1',1:'md2',2:'md3',3:'fw1',4:'fw2',5:'fw3',6:'fw4',7:'fw5'})
        fw_df['user_id'] =fw_df['user_id']+10000

        pre_df=fw_df.iloc[:,:9]

        md_flow = {str(x.astype("int")):(i+1) for i,x in enumerate(
            pre_df[['md1','md2','md3']].drop_duplicates().reset_index(drop=True).values)}
        
        fw_flow = {str(x):(i+1) for i,x in enumerate(
            pre_df[['fw1','fw2','fw3','fw4','fw5']].drop_duplicates().reset_index(drop=True).values)}


        def fw_change_counter(x):# 버전 변화 횟수
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[x[v]]
            if len(fwlst) ==len(list(set(fwlst))):
                return 0
            else:
                return 1

        def mean_str_fw_dum(x):# 평균
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[int(x[v].replace('.',""))]
            return np.array(fwlst).mean()

        def std_str_fw_dum(x): # 표준편차
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[int(x[v].replace('.',""))]
            return np.array(fwlst).std()

        pre_df=fw_df.iloc[:,:9]
        pre_df['md_counts'] = pre_df[['md1','md2','md3']].astype('bool').sum(axis=1)
        pre_df['fw_counts'] = pre_df[['fw1','fw2','fw3','fw4','fw5']].astype('bool').sum(axis=1)

        pre_df['fw_change'] = pre_df.apply(fw_change_counter,axis=1)
        pre_df['fw_flows'] = pre_df.apply(lambda x : fw_flow[str(x[['fw1','fw2','fw3','fw4','fw5']].values)],axis=1)
        pre_df['md_flows'] = pre_df.apply(lambda x :md_flow[str(x[['md1','md2','md3']].values.astype("int"))],axis=1)
        pre_df['fw_mean'] = pre_df.apply(mean_str_fw_dum,axis=1)
        pre_df['fw_std'] = pre_df.apply(std_str_fw_dum,axis=1)


        fw_model_flow =pre_df.iloc[:,9:].values 
        return fw_model_flow
    
    def get_time_term(self):
        df = self.df
        first_num = self.first
        count_num = self.num_user

        time_term = np.zeros((count_num,4))
        tmp =df[['user_id','time']].drop_duplicates()

        for v in tqdm(tmp.user_id.unique()):
            test =tmp.loc[tmp.user_id ==v].time
            if len(test) <=2:
                time_term[v-first_num,0] = 0
                time_term[v-first_num,1] = 0
                time_term[v-first_num,2] = test.values[-1]-test.values[0]
                time_term[v-first_num,3] = len(test)
            else:
                time_term[v-first_num,0] = (test -test.shift(1)).max().total_seconds()
                time_term[v-first_num,1] = (test -test.shift(1)).min().total_seconds()
                time_term[v-first_num,2] = test.values[-1]-test.values[0]  
                time_term[v-first_num,3] = len(test)

        dft = pd.DataFrame(time_term).copy()

        dft[0] =dft[0]/3600
        dft[2] =dft[2]/3600/24/10e8
        dft[2] =np.where(dft[2].values==0,1,dft[2].values)
        dft[5] =dft[0]/dft[3]
        dft[5] = dft[0]/dft[3]*3600
        dft[6] = dft[0]/24/dft[2]
        return dft.fillna(0).values
        
    def connetion_err(self):
        first_num = self.first
        count_num = self.num_user
        
        err_df =self.df[['user_id','time','errcode']].dropna(axis=0)

        df_con =err_df.loc[err_df.errcode.str.contains("connection")]
        df_con['check_time'] = df_con.time.dt.date
        df_con['check_hour'] =df_con.time.dt.hour

        def connetion_df(con_df):
            day_con_err =con_df[['user_id','check_time']].value_counts().groupby("user_id").max()
            hour_con_err=con_df.groupby(['user_id','check_time','check_hour']).size().groupby("user_id").max()
            tenmin_con_err =con_df.set_index('time').groupby(['user_id','errcode']).resample("10min").size().groupby("user_id").max()
            con_trans =pd.concat([day_con_err,hour_con_err,tenmin_con_err],axis=1)
            return con_trans

        total_conn_err=[connetion_df(df_con)]
        for errs in tqdm(['connection timeout', 'connection fail to establish','connectionterminated by local host',
                          'connection fail for LMP response timout','L2CAP connection cancelled']):
            
            con_esta = df_con.loc[df_con.errcode.str.contains(errs)]
            total_conn_err.append(connetion_df(con_esta))

        base_df =pd.DataFrame(range(first_num,first_num+count_num)).rename(columns={0:'user_id'}).set_index('user_id')

        connetion_err_pre = pd.concat(total_conn_err+[base_df],axis=1).fillna(0).values
        return connetion_err_pre
    
    def fwver_time(self):
        
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
            
        tsed = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        dfw = tsed[['user_id','fwver']]
        fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

        main_fw_ar = np.zeros((self.num_user,6))
        for i,tgid in enumerate(tqdm(range(self.first,self.first+self.num_user))):

            tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
            tgidtotal = tsed.loc[tsed.user_id ==tgid]
            
            try:
                data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
                t1 =data.time
                time_delta = (t1-t1.shift(1)).dt.total_seconds()

                main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]
                main_fw_ar[i,0] = fwver_total_dic[main_fwver]
                main_fw_ar[i,1] =(time_delta[1::2].values).max().astype('float')/(time_delta.values[1:]).sum().astype('float')  #target fw workingtime / total
                if len(time_delta) ==1:
                    main_fw_ar[i,2] =0  
                    main_fw_ar[i,3] =0  
                    main_fw_ar[i,4] =0  
                    main_fw_ar[i,5] =0  
                else:
                    main_fw_ar[i,2] =time_delta[::2].min()/3600 
                    main_fw_ar[i,3] =time_delta[::2].std()/3600
                    main_fw_ar[i,4] =time_delta[1::2].values.astype('float').std()/3600 
                    main_fw_ar[i,5] =(time_delta[1::2].values.astype('float')/3600).var()  
            except:
                main_fw_ar[i,0] =0
                main_fw_ar[i,1] =0
                main_fw_ar[i,2] =0
                main_fw_ar[i,3] =0
                main_fw_ar[i,4] =0
                main_fw_ar[i,5] =0
                
        return main_fw_ar
    
    def fw_err_cnt(self):
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
            
        tred = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        tred_all = self.df.dropna(axis=0).reset_index(drop=True)
        
        tsed = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        dfw = tsed[['user_id','fwver']]
        fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

        main_fw_err_counts = np.zeros((self.num_user,84))
        for i,tgid in enumerate(tqdm(range(self.first,self.first+self.num_user))):

            tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
            tgidtotal = tred_all.loc[tred_all.user_id ==tgid]

            
            try:
                data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
                t1 =data.time
                time_delta = (t1-t1.shift(1)).dt.total_seconds()

                main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]


                main_date = tgidtotal.loc[tgidtotal.fwver ==main_fwver][['time','errtype']]
                main_date['date'] =main_date.time.dt.date
                main_time_del  =(main_date.time.dt.date.values[-1] -main_date.time.dt.date.values[0]).days
                date_first= main_date.time.dt.date.values[0]
                err_count_main = np.zeros((42,main_time_del+1))
                for n in  range(main_time_del+1):
                    lcdf = main_date.loc[main_date.date ==date_first]
                    for errtype in lcdf.errtype.values:
                        err_count_main[errtype-1,n] +=1
                    date_first += dt.timedelta(days=1)

                main_fw_err_counts[i,:42] =     err_count_main.mean(axis=1)
                main_fw_err_counts[i,42:] =     err_count_main.std(axis=1)

            except:
                main_fw_err_counts[i,:] = 0
                
        return main_fw_err_counts

In [ ]:
def get_col_names (df):
    col_dict = dict()
    
    for i in range(5) : # 펌웨어 버전 변화
        col_dict[str(i)] = 'fw' + str(i+1)  
    for i in range(5,7):# 모델 변화
        col_dict[str(i)] = 'model' + str(i-4)
        
    for i in range(49,91) :# 주별 에러타입 평균
        col_dict[str(i)] = 'mean_week_errtype' + str(i-48)
          
    for i in range(133,175): # 시간당 에러타입 평균
        col_dict[str(i)] = 'mean_hour_errtype' + str(i - 132)
    for i in range(175,217): # 시간당 에러타입 표준편차
        col_dict[str(i)] = 'std_hour_errtype' + str(i - 174)
    for i in range(217,259): # 시간당 에러타입 최댓값
        col_dict[str(i)] = 'max_hour_errtype' + str(i - 216)
        
    for i in range(259,301): # 일별 에러타입 평균
        col_dict[str(i)] = 'mean_day_errtype' + str(i - 260)
    for i in range(301,343): # 일별 에러타입 최댓값
        col_dict[str(i)] = 'max_day_errtype' + str(i - 133)
    for i in range(343,385): # 일별 에러타입 표준편차
        col_dict[str(i)] = 'std_day_errtype' + str(i - 342)
    
    for i in range(385,392): # 펌웨어 버전 흐름
        col_dict[str(i)] = 'fw_ver_flow' + str(i - 384)
    for i in range(392,398): # 버전 사용 기간
        col_dict[str(i)] = 'fw_ver_term' + str(i - 391)
    for i in range(398,416): # connection errcode
        col_dict[str(i)] = 'connection_' + str(i - 397)
        
    for i in range(416,422): # 주 사용 fwver
        col_dict[str(i)] = 'main_ver_' + str(i - 415)
        
    for i in range(422,506): # fw_errtype
        col_dict[i] = 'fw_errtype' + str(i - 421)
    return df.rename(columns = col_dict)
    

In [ ]:
train5 = mk_dataset3(train_err2,'train',15000,42,10000).run()
train5 = pd.DataFrame(np.concatenate(tuple(train5),axis=1))

In [ ]:
test5 = mk_dataset3(test_err2,'test',14999,42,30000).run()
test5 = pd.DataFrame(np.concatenate(tuple(test5),axis=1))

## 2-4. 최종 데이터 병합

In [ ]:
def select_concat(df1,df2,df3,df4,df5) :# 변수 선택 및 데이터프레임 병합
    select1 = df5.iloc[ : , 0:8]
    select2 = df5.iloc[ : , 50:92]
    select3 = df5.iloc[ : , 134:]

    final_df = pd.concat([df1, df2, df3, df4,
                         select1, select2, select3], axis=1)
    
    for i in final_df.columns:
        str_i = str(i)
        if 'diff' in str_i :
            final_df.drop(i, axis='columns', inplace=True)
    return final_df

In [ ]:
train_final = select_concat(train1, train2, train3, train4 ,train5)
test_final = select_concat(test1, test2, test3,test4, test5)

train_final.to_csv(SAVE_PATH + "./trainf.csv", index = False)
test_final.to_csv(SAVE_PATH + "./testf.csv", index = False)

# 모델 훈련 및 예측

## 3-1. 모델학습

In [ ]:
print(train_final.isnull().sum().sum())
print(train_final.isnull().sum().sort_values(ascending=False))

In [ ]:
# train_final = pd.read_csv(SAVE_PATH + "./trainf.csv")
# test_final = pd.read_csv(SAVE_PATH + "./testf.csv")

# 결측치가 많은 칼럼 처리
del train_final[419]
del train_final[418]
train_final[417] = train_final[417].fillna(1.000000)

del test_final[419]
del test_final[418]
test_final[417] = test_final[417].fillna(1.000000)
test_final = test_final.fillna(0)

In [ ]:
# y값 생성
label = np.zeros(15000)
label[train_problem.user_id.unique()-10000] = 1 

In [ ]:
train_x = train_final.copy()
train_y = label.astype(int)

# 카테고리 변수 지정
train_x['ver'] = train_x['ver'].astype('category')
train_x['ver']= train_x['ver'].cat.codes

test_final['ver'] = test_final['ver'].astype('category')
test_final['ver']= test_final['ver'].cat.codes

(train_x.shape, train_y.shape)

## 듀얼 블렌딩
  - 1. 주요 파라미터 및 최적값들을 찾음.
  - 2. 두 최적값 각각을 사용하는 두 모델을 생성 후 학습
  - 3. 최종 예측 결과값을 블렌딩

In [ ]:
# 파라미터가 다른 두 캣부스트 모델을 동시에 학습후 블렌딩

def dual_blending (train_x, train_y):
    
    models1    = []
    auc_scores1   = []
    models2    = []
    auc_scores2   = []

    threshold = 0.5

    # 5 Kfold cross validation
    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in k_fold.split(train_x,train_y):

        X = train_x.iloc[train_idx,:]
        y = train_y[train_idx]
        valid_x = train_x.iloc[val_idx,:]
        valid_y = train_y[val_idx]

        # 첫번째 모델
        model1 = CatBoostRegressor(iterations=1000, l2_leaf_reg = 0.01, random_seed = 1422)
        model1.fit(X,y, cat_features = ['ver'])


        valid_prob1 = model1.predict(valid_x)
        valid_pred1 = np.where(valid_prob1 > threshold, 1, 0)
        auc_score1 = roc_auc_score(   valid_y, valid_prob1)


        models1.append(model1)
        auc_scores1.append(auc_score1)

        print('==========================================================')

        # 두번째 모델
        model2 = CatBoostRegressor(iterations=1000, l2_leaf_reg = 0.003, random_seed = 1422)
        model2.fit(X,y, cat_features = ['ver'])


        valid_prob2 = model2.predict(valid_x)
        valid_pred2 = np.where(valid_prob2 > threshold, 1, 0)
        auc_score2 = roc_auc_score(valid_y, valid_prob2)


        models2.append(model2)
        auc_scores2.append(auc_score2)

        print('==========================================================')
        
    return models1, models2, auc_scores1, auc_scores2

In [ ]:
group1, group2, scores1, scores2 = dual_blending(train_x, train_y)

## 예측 및 제출파일 생성

In [ ]:
# 예측
pred_y_list1 = []
pred_y_list2 = []

for m1 in group1:
    pred_y1 = m1.predict(test_final)
    pred_y_list1.append(pred_y1.reshape(-1,1))
    
for m2 in group2:    
    pred_y2 = m2.predict(test_final)
    pred_y_list2.append(pred_y2.reshape(-1,1))
    
pred1 = np.mean(pred_y_list1, axis = 0)
pred2 = np.mean(pred_y_list2, axis = 0)
(pred1,pred2)

In [ ]:
pred_ensemble = 0.5 * pred1 + 0.5 * pred2
pred_ensemble

sample_submssion = pd.read_csv(DATA_PATH + 'sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)
sample_submssion.to_csv(RESULT_PATH + "Final_Sub.csv", index = False)
sample_submssion